# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. <font color='blue'>Предобработка данных не понадобится — вы её уже сделали.</font>

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn import tree



df = pd.read_csv('/datasets/users_behavior.csv')
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

    сalls — количество звонков,
    minutes — суммарная длительность звонков в минутах,
    messages — количество sms-сообщений,
    mb_used — израсходованный интернет-трафик в Мб,
    is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разделим исходные данные на обучающую(train), валидационную(validate) и тестовую выборки(test).

Разделим датасет с помощью метода split() из numpy.  
Обучающая выборка - 60%(int(.6*len(df))), 
следующая выборка валидационная - 20%( int(.8*len(df)) ) 
и оставщаяся доля датасета - тестовая выборка - 20%. 


In [4]:
# 60% отводим под обучающую выборку
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
# половину из оставшихся 40% отдаем на валидационную, другую половину – на тестовую
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 3027 to 482
Data columns (total 5 columns):
calls       1928 non-null float64
minutes     1928 non-null float64
messages    1928 non-null float64
mb_used     1928 non-null float64
is_ultra    1928 non-null int64
dtypes: float64(4), int64(1)
memory usage: 90.4 KB


In [6]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 1386 to 3197
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 160 to 2313
Data columns (total 5 columns):
calls       643 non-null float64
minutes     643 non-null float64
messages    643 non-null float64
mb_used     643 non-null float64
is_ultra    643 non-null int64
dtypes: float64(4), int64(1)
memory usage: 30.1 KB


## Исследуйте модели

Для решения задачи будем использовать три модели: *дерево решений, случайный лес и линейная регрессия*. 

Определим признаки для всех выборок. 

In [8]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

### Дерево решений.

In [9]:
model_DT = None
best_result_DT = 0
best_depth_DT = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result_DT:
        model_DT = model
        best_result_DT = result
        best_depth_DT = depth
        
print(f"Accuracy лучшей модели: {best_result_DT}, глубина дерева: {best_depth_DT}")

Accuracy лучшей модели: 0.7853810264385692, глубина дерева: 3


### Случайный лес.

C количеством деревьев: от 10 до 50 с шагом 10,
C максимальной глубиной от 1 до 10.

In [10]:
model_RFC = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            model_RFC = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy лучшей модели:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)

Accuracy лучшей модели: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 8


### Линейная регрессия. 

In [11]:
model_LR = LogisticRegression(random_state=12345)
model_LR.fit(features_train, target_train)
result = model_LR.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Лучший результат показалал алгоритм случайного леса:  

    аccuracy лучшей модели - 0.8087091757387247  
    количество деревьев - 40  
    максимальная глубина - 8

## Проверьте модель на тестовой выборке

Дерево решений. 

In [12]:
predictions_test_DT = model_DT.predict(features_test)
accuracy_score(target_test, predictions_test_DT)

0.7791601866251944

Случайный лес

In [13]:
predictions_test_RFC = model_RFC.predict(features_test)
accuracy_score(target_test, predictions_test_RFC)

0.7962674961119751

Логистическая регрессия. 

In [14]:
predictions_test_LR = model_LR.predict(features_test)
accuracy_score(target_test, predictions_test_LR)

0.7402799377916018

**Вывод:**
на тестовой выборке все наши модели показывают приближенное к тренировочной выборке значения accuracy. Явной проблемы с переобучением не наблюдается, поэтому можем считать, что нам удалось построить модель для задачи бинарной классификации, которая выберет подходящий тариф.


## (бонус) Проверьте модели на адекватность

Большинство алгоритмов машинного обучения работают лучше всего, когда количество образцов в каждом классе примерно одинаково. Это потому, что большинство алгоритмов предназначены для максимизации точности и уменьшения ошибок.  

Наши данные по тарифам распределены неравномерно(«Ультра» — 1, «Смарт» — 0), поэтому следует проверить результат обучения модели фиктивным классификатором DummyClassifier. Ожидается, что любой другой классификатор будет работать лучше для данного набора данных. Это особенно полезно для наборов данных, где есть уверенность в дисбалансе классов.


In [15]:
df.is_ultra.value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Применим две стратегии: **most_frequent** и **stratified**.

- **most_frequent**: классификатор всегда предсказывает самый частый ярлык класса в данных обучения
- **stratified**: генерирует прогнозы, соблюдая распределение классов по тренировочным данным. Он отличается от «наиболее частой» стратегии, поскольку вместо этого ассоциирует вероятность того, что каждая точка данных является наиболее частой меткой класса

In [17]:
dummy_clf = DummyClassifier(strategy="most_frequent" ,random_state=12345)
dummy_clf.fit(features_train, target_train)
accuracy_dummy = dummy_clf.score(features_test, target_test)
print("Качество фиктивного классификатора:", accuracy_dummy)


Качество фиктивного классификатора: 0.6842923794712286


In [18]:
dummy_clf = DummyClassifier(strategy="stratified" ,random_state=12345)
dummy_clf.fit(features_train, target_train)
accuracy_dummy = dummy_clf.score(features_test, target_test)
print("Качество фиктивного классификатора:", accuracy_dummy)

Качество фиктивного классификатора: 0.536547433903577


Качество фиктивный классификатора ниже наших обученных моделей, что и требовалось доказать. 